In [1]:
# /código em Python que lê dados do HDFS e grava em Tabelas Iceberg em um amibiente de Big Data.

In [4]:
!pip install iceberg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.8/159.8 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.5/480.5 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.8 kB 27.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created 

In [5]:
!pip install pyarrow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyarrow import fs
from pyarrow.dataset import dataset
from pyarrow import Table
from pyarrow import schema
#from pyarrow import iceberg

In [ ]:
# Configurando o Cliente do HDFS
hdfs = fs.HadoopFileSystem(host='dominio.com.br', port=8020, user='hadoop_user', passwd='senhas_user')

In [ ]:
# Caminho par aos dados de origemno HDFS
source_data_path = '/path/to/source_data'

In [ ]:
# Configura o local de destino no Arquivo ICeberg.
iceberg_table_location = 'hdfs:/home/data_lakehouse/INSS/dados_saida/'

In [ ]:
# Criando Objeto Iceberg Table
iceberg_table = iceberg.Table.load(iceberg_table_location, hdfs=hdfs)

In [ ]:
# Definir o esquema da tabela Iceberg
schema = schema.Schema.from_pandas(df) # Substitua 'df' pelo DataFrame corrente.

In [ ]:
# Criar um novo arquivo Iceberg
new_file = iceberg_table.new_data_file()

In [ ]:
# Configurar o escritor Iceberg
with new_file.new_row_writer() as writer:
  # Ler os dados de origem do HDFS
  dataset = dataset(source_data_path, format='parquet', filesystem=hdfs)

  for batch in dataset.to_batches():
    # Converter o lote de dados para o foramto Arrow Table
    arrow_table = Table.from_batches([batch], schema=schema)

    # Gravar os dados na tabela Iceberg
    writer.write_table(arrow_table)

In [ ]:
# Adicionar o novo arquivo à Tabela Iceberg
iceberg_table.new_version().add_file(new_file).commit()